In [1]:
import itertools
import os 
#os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

import matplotlib.pylab as plt
import numpy as np

import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras.preprocessing.image import ImageDataGenerator

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

print("TF version:", tf.__version__)
print("Hub version:", hub.__version__)
print("GPU is", "available" if tf.test.is_gpu_available() else "NOT AVAILABLE")


/home/IBEO.AS/soe/UMLA/virtualenv/tensorflow_2_1/lib/python3.6/site-packages/pandas/compat/__init__.py:117: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


1 Physical GPUs, 1 Logical GPUs
TF version: 2.1.0
Hub version: 0.10.0
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
GPU is available


In [2]:
img_height = 224
img_width = 224
batch_size = 12

train_generator = ImageDataGenerator(rescale=1. / 255,
                                     horizontal_flip=True,
                                     validation_split=0.1)

test_generator = ImageDataGenerator(rescale=1. / 255,
                                    validation_split=0.1)

ds_train = train_generator.flow_from_directory(directory='dataset/rock_paper_scissor_videobased/',
                                               batch_size=batch_size,
                                               shuffle=True,
                                               target_size=(img_height, img_width),
                                               seed=42,
                                               classes=["rock", "paper", "scissor"],
                                               subset="training")

ds_test = test_generator.flow_from_directory(directory='dataset/rock_paper_scissor_videobased/',
                                             batch_size=batch_size,
                                             shuffle=False,
                                             target_size=(img_height, img_width),
                                             seed=42,
                                             classes=["rock","paper", "scissor"],
                                             subset="validation")


Found 4595 images belonging to 3 classes.
Found 509 images belonging to 3 classes.


In [3]:
module_selection = ("mobilenet_v2_100_224", 224)
handle_base, pixels = module_selection
MODULE_HANDLE ="https://tfhub.dev/google/imagenet/{}/feature_vector/4".format(handle_base)
IMAGE_SIZE = (pixels, pixels)
print("Using {} with input size {}".format(MODULE_HANDLE, IMAGE_SIZE))

BATCH_SIZE = batch_size

Using https://tfhub.dev/google/imagenet/mobilenet_v2_100_224/feature_vector/4 with input size (224, 224)


In [4]:
print("Building model with", MODULE_HANDLE)
model = tf.keras.Sequential([
    # Explicitly define the input shape so the model can be properly
    # loaded by the TFLiteConverter
    tf.keras.layers.InputLayer(input_shape=IMAGE_SIZE + (3,)),
    hub.KerasLayer(MODULE_HANDLE, trainable=True),
    tf.keras.layers.Dropout(rate=0.2),
    tf.keras.layers.Dense(3, kernel_regularizer=tf.keras.regularizers.l2(0.0001)),
    tf.keras.layers.Activation(activation='softmax')
])
model.build((None,)+IMAGE_SIZE+(3,))
model.summary()

Building model with https://tfhub.dev/google/imagenet/mobilenet_v2_100_224/feature_vector/4
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 1280)              2257984   
_________________________________________________________________
dropout (Dropout)            (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 3)                 3843      
_________________________________________________________________
activation (Activation)      (None, 3)                 0         
Total params: 2,261,827
Trainable params: 2,227,715
Non-trainable params: 34,112
_________________________________________________________________


In [5]:
model.compile(
  optimizer=tf.keras.optimizers.SGD(lr=0.005, momentum=0.9), 
  loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False, label_smoothing=0.1),
  metrics=['accuracy'])

In [6]:
steps_per_epoch = ds_train.samples // ds_train.batch_size
validation_steps = ds_test.samples // ds_test.batch_size
hist = model.fit(
    ds_train,
    epochs=3, steps_per_epoch=steps_per_epoch,
    validation_data=ds_test,
    validation_steps=validation_steps).history

  ...
    to  
  ['...']


  ...
    to  
  ['...']


  ...
    to  
  ['...']


  ...
    to  
  ['...']


Train for 382 steps, validate for 42 steps
Epoch 1/3
382/382 [==============================] - 228s 596ms/step - loss: 0.5557 - accuracy: 0.9236 - val_loss: 0.4645 - val_accuracy: 0.9623
Epoch 2/3
382/382 [==============================] - 220s 577ms/step - loss: 0.4124 - accuracy: 0.9902 - val_loss: 0.4406 - val_accuracy: 0.9603
Epoch 3/3
382/382 [==============================] - 220s 575ms/step - loss: 0.3918 - accuracy: 0.9978 - val_loss: 0.4357 - val_accuracy: 0.9623


In [7]:
saved_model_path = "/tmp/saved_flowers_model3"
tf.saved_model.save(model, saved_model_path)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


INFO:tensorflow:Assets written to: /tmp/saved_flowers_model3/assets


INFO:tensorflow:Assets written to: /tmp/saved_flowers_model3/assets


In [8]:
# Fetch the Keras session and save the model
# The signature definition is defined by the input and output tensors,
# and stored with the default serving key
import tempfile

MODEL_DIR = tempfile.gettempdir()
version = 2
export_path = os.path.join(MODEL_DIR, str(version))
print('export_path = {}\n'.format(export_path))

tf.keras.models.save_model(
    model,
    export_path,
    overwrite=True,
    include_optimizer=True,
    save_format=None,
    signatures=None,
    options=None
)

print('\nSaved model:')
!ls -l {export_path}


export_path = /tmp/2

INFO:tensorflow:Assets written to: /tmp/2/assets


INFO:tensorflow:Assets written to: /tmp/2/assets



Saved model:
total 3128
drwxr-xr-x 2 soe domain users    4096 Nov  8 16:36 assets
-rw-r--r-- 1 soe domain users 3191961 Nov  8 16:36 saved_model.pb
drwxr-xr-x 2 soe domain users    4096 Nov  8 16:36 variables


In [9]:
import cv2

rock_paper_scissor_labels = ["Rock", "Paper", "Scissor"]

frame = cv2.imread("/home/IBEO.AS/soe/Documents/GitHub/Image-Classification-App/rock_paper_scissor_classification/dataset/validation/images/2020-11-04 19:24:42.866460.png")

resized = cv2.resize(frame, (224, 224), interpolation = cv2.INTER_AREA)

resized = resized / 255

input_image = np.expand_dims(resized, axis=0)

result = model.predict(input_image)
print(rock_paper_scissor_labels[np.argmax(result)])

Rock
